In [39]:
# Reading json file to understand its structure
import json
from pandas.io.json import json_normalize

# Converting Train data into a dataframe
with open('train.json') as train_file:
    dict_train = json.load(train_file)
    df_train = pd.io.json.json_normalize(dict_train)

# Extracting Required fields
df_train = df_train[['request_id', 'request_title', 
               'request_text_edit_aware', 
               'requester_received_pizza']]

print df_train.head()
print '---------------------------------------------------------------'

# Converting Test data into a dataframe
with open('test.json') as test_file:
    dict_test = json.load(test_file)
    df_test = pd.io.json.json_normalize(dict_test)

# Extracting Required fields
df_test = df_test[['request_id', 'request_title', 
               'request_text_edit_aware']]

print df_test.head()

  request_id                                      request_title  \
0   t3_l25d7            Request Colorado Springs Help Us Please   
1   t3_rcb83  [Request] California, No cash and I could use ...   
2   t3_lpu5j  [Request] Hungry couple in Dundee, Scotland wo...   
3   t3_mxvj3  [Request] In Canada (Ontario), just got home f...   
4  t3_1i6486  [Request] Old friend coming to visit. Would LO...   

                             request_text_edit_aware  requester_received_pizza  
0  Hi I am in need of food for my 4 children we a...                     False  
1  I spent the last money I had on gas today. Im ...                     False  
2  My girlfriend decided it would be a good idea ...                     False  
3  It's cold, I'n hungry, and to be completely ho...                     False  
4  hey guys:\n I love this sub. I think it's grea...                     False  
---------------------------------------------------------------
  request_id                                   